In [1]:
import sqlite3
import pandas as pd
import os
import pickle


# Why Python Pickle is used?
The pickle module in Python is used for serializing and deserializing Python objects. Serialization is the process of converting a Python object into a byte stream, which can then be stored in a file, sent over a network, or saved in a database. Deserialization is the reverse process, where the byte stream is converted back into a Python object.

In [5]:
# In this block we make a new table in the database from the csv file

import subprocess

# Path to the CSV file and database
csv_file = 'data/cars_filtered_ml.csv'
db_file = 'data/cars_filtered_ml.db'
table_name = 'my_table'

# Command to import CSV into SQLite
command = f'sqlite3 {db_file} ".mode csv" ".import {csv_file} {table_name}"'

# Execute the command
subprocess.run(command, shell=True, check=True)

CompletedProcess(args='sqlite3 data/cars_filtered_ml.db ".mode csv" ".import data/cars_filtered_ml.csv my_table"', returncode=0)

In [6]:
# Function to read image from path and convert to binary and serialize it to store in the database
def images_to_binary(images_dir):
    binary_images = []
    for image_path in os.listdir(images_dir):
        image_path = os.path.join(images_dir, image_path)
        with open(image_path.strip(), 'rb') as file:
            binary_images.append(file.read())
    return pickle.dumps(binary_images)  # Serialize the list of binary images


In [ ]:
df = pd.read_csv(csv_file)

# Add a new column to the dataframe with the binary images 
df['images'] = df['images_path'].apply(images_to_binary)

# Connessione al database SQLite (crea il database se non esiste)
conn = sqlite3.connect(db_file)

# Save the dataframe to a SQLite table
df.to_sql(table_name, conn, if_exists='replace', index=False)

# Commit the changes
conn.commit()
conn.close()

C:\Users\sebas\AppData\Local\Temp\ipykernel_2804\2269995369.py:1: DtypeWarning: Columns (26,27,28,29,34,48,49,50,52,81,86,96,97,98,99,100,101,102,103,104,106,114,115,116,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,135,137,138,139,142,144,148,149,150,152,153,154,155,156,157,158,159,160,162,163,164,165,166,169,170,177,178,179,185,186,187,188,190,195,196,202) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(csv_file)


FileNotFoundError: [WinError 3] Impossibile trovare il percorso specificato: 'data/catalog_method/images/Abarth/124 Spider/124 GT'

In [18]:
def save_first_image(serialized_images, output_dir='output_images', output_filename='retrieved_image.jpg'):
    # Deserialize the images
    images = pickle.loads(serialized_images)
    
    # Ensure the output directory exists
    os.makedirs(output_dir, exist_ok=True)
    
    # Define the output path
    output_path = os.path.join(output_dir, output_filename)
    
    # Save the first image to a file
    with open(output_path, 'wb') as file:
        file.write(images[0])
    
    print(f'Image saved to {output_path}')

In [23]:
# Create a connection to the SQLite database
conn = sqlite3.connect(db_file)

# Read the data from the table into a pandas DataFrame
df_solo = pd.read_sql_query(f'SELECT * FROM {table_name}', conn)


# Close the connection
conn.close()

imgs = df['images'][1]

save_first_image(imgs)

Image saved to output_images\retrieved_image.jpg
